<a href="https://colab.research.google.com/github/handsome1201/CNN_pneumonia/blob/main/%ED%8F%90%EC%A7%84%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

데이터 가져오기

In [ ]:
train_file_0= os.listdir("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/train/NORMAL")
train_file_1= os.listdir("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/train/PNEUMONIA")
test_file_0= os.listdir("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/test/NORMAL")
test_file_1= os.listdir("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/test/PNEUMONIA")

train데이터 세팅

In [ ]:
X_full= 0
check= False

for file in train_file_0:
    img_in= cv2.imread("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/train/NORMAL/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    
    if(check== False):
        X_full= img
        check= True
    else:
        X_full= np.vstack((X_full, img))

for file in train_file_1:
    img_in= cv2.imread("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/train/PNEUMONIA/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    X_full= np.vstack((X_full, img))

In [ ]:
y_full= np.array([0]* len(train_file_0))
y_full_1= np.array([1]* len(train_file_1))
y_full= np.append(y_full, y_full_1)

In [ ]:
X_test, y_train= shuffle(X_full, y_full, random_state= 42)

traiin 데이터와 val 데이터 분리

In [ ]:
X_train, X_val, y_train, y_val= train_test_split(X_full, y_full, test_size= 0.25)

데이터 저장

In [ ]:
np.save("/content/drive/MyDrive/폐질환/X_train", X_train)
np.save("/content/drive/MyDrive/폐질환/Y_train", y_train)
np.save("/content/drive/MyDrive/폐질환/X_val", X_val)
np.save("/content/drive/MyDrive/폐질환/Y_val", y_val)

In [ ]:
X_test= 0
check= False

for file in test_file_0:
    img_in= cv2.imread("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/test/NORMAL/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    
    if(check== False):
        X_test= img
        check= True
    else:
        X_test= np.vstack((X_test, img))

for file in test_file_1:
    img_in= cv2.imread("/content/drive/MyDrive/폐질환/archive/chest_xray"+ "/test/PNEUMONIA/"+ file, cv2.IMREAD_GRAYSCALE)
    img_res= cv2.resize(img_in, dsize= (100, 100), interpolation= cv2.INTER_LINEAR)
    img_round= img_res/255
    img_resh= img_round.reshape((100, 100, 1))
    img= img_resh.reshape((1, )+ img_resh.shape)
    X_test= np.vstack((X_test, img))

In [ ]:
y_test= np.array([0]* len(test_file_0))
y_test_1= np.array([1]* len(test_file_1))
y_test= np.append(y_test, y_test_1)

In [ ]:
X_test, y_test= shuffle(X_test, y_test, random_state= 42)

데이터 저장

In [ ]:
np.save("/content/drive/MyDrive/폐질환/X_test", X_test)
np.save("/content/drive/MyDrive/폐질환/Y_test", y_test)

데이터 불러오기

In [ ]:
X_train= np.load("/content/drive/MyDrive/폐질환/X_train.npy")
y_train= np.load("/content/drive/MyDrive/폐질환/Y_train.npy")
X_val= np.load("/content/drive/MyDrive/폐질환/X_val.npy")
y_val= np.load("/content/drive/MyDrive/폐질환/Y_val.npy")
X_test= np.load("/content/drive/MyDrive/폐질환/X_test.npy")
y_test= np.load("/content/drive/MyDrive/폐질환/Y_test.npy")

모양 확인

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(3912, 100, 100, 1)
(3912,)
(1304, 100, 100, 1)
(1304,)
(624, 100, 100, 1)
(624,)


모델 생성

In [ ]:
model= keras.models.Sequential()

In [ ]:
model.add(keras.layers.Conv2D(64, 10, activation= "relu", padding= "same", input_shape= [100, 100, 1]))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(128, 4, activation= "relu", padding= "same"))
model.add(keras.layers.Conv2D(128, 4, activation= "relu", padding= "same")) 
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(256, 4, activation= "relu", padding= "same"))
model.add(keras.layers.Conv2D(256, 4, activation= "relu", padding= "same"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation= "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(64, activation= "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation= "relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation= "softmax"))

In [ ]:
model.compile(loss= "sparse_categorical_crossentropy", optimizer= "sgd", metrics= ["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      6464      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 128)       131200    
                                                                 
 conv2d_2 (Conv2D)           (None, 50, 50, 128)       262272    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 25, 128)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 25, 25, 256)       5

In [ ]:
start_time= time.time()
history= model.fit(X_train, y_train, epochs= 30, validation_data= (X_val, y_val))
end_time= time.time()
print("time: ", end_time- start_time)

Epoch 1/30
123/123 [==============================] - 1175s 10s/step - loss: 0.6120 - accuracy: 0.7247 - val_loss: 0.5671 - val_accuracy: 0.7615
Epoch 2/30
123/123 [==============================] - 1163s 9s/step - loss: 0.5813 - accuracy: 0.7365 - val_loss: 0.5046 - val_accuracy: 0.7615
Epoch 3/30
123/123 [==============================] - 1161s 9s/step - loss: 0.5269 - accuracy: 0.7352 - val_loss: 0.4165 - val_accuracy: 0.7615
Epoch 4/30
123/123 [==============================] - 1157s 9s/step - loss: 0.4560 - accuracy: 0.7372 - val_loss: 0.3200 - val_accuracy: 0.7615
Epoch 5/30
123/123 [==============================] - 1161s 9s/step - loss: 0.3874 - accuracy: 0.7339 - val_loss: 0.2954 - val_accuracy: 0.7615
Epoch 6/30
123/123 [==============================] - 1153s 9s/step - loss: 0.3788 - accuracy: 0.7794 - val_loss: 0.4004 - val_accuracy: 0.7623
Epoch 7/30
123/123 [==============================] - 1166s 9s/step - loss: 0.3512 - accuracy: 0.8474 - val_loss: 0.4190 - val_accuracy

모델 저장

In [ ]:
model.save("./model/learning_model")

In [ ]:
model= load_model("./model/learning_model")

손실 정확도 확인

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pd.DataFrame(history.history).plot(figsize= (8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

테스트 데이터 확인

In [ ]:
temp=model.predict(X_test)
y_pred= np.argmax(temp, axis=1)

정확도 확인

In [ ]:
accuracy= 0

for i in range(len(y_test)):
    if(y_test[i]== y_pred[i]):
        accuracy+= 1
        
print("accuracy: ", accuracy/len(y_test))